# <font color='black'><div align='center'>wiki4HE Data Set</div></font>
# <font color='blue'>Survey of faculty members from two Spanish universities on teaching uses of Wikipedia</font>

# Preparação do ambiente

In [1]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

plotly.offline.init_notebook_mode(connected = True)

In [2]:
# Spark Session - usada quando se trabalha com DataFrames no Spark
spSession = SparkSession.builder.master("local").appName("DSA-SparkSQL").config("spark.some.config.option","some-value").getOrCreate()

# Preparação dos dados

Foi criado um arquivo csv (**attributeInformation**) que mapeia alguns atributos do Data Set, de modo a auxiliar a apresentação
descritiva dos atributos.

A descrição completa da base pode ser lida em: <a href="archive.ics.uci.edu/ml/datasets/wiki4HE">archive.ics.uci.edu/ml/datasets/wiki4HE</a>

In [3]:
# Carregando arquivos
wiki4HE_DF = spSession.read.option("sep", ";").csv("wiki4HE.csv", header = True)
attributeInformation_DF = spSession.read.option("sep", ";").csv("attributeInformation.csv", header = True)

In [4]:
# Exibe o esquema do Data Set Carregado, com todas as variáveis disponíveis
wiki4HE_DF.printSchema()

root
 |-- AGE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- DOMAIN: string (nullable = true)
 |-- PhD: string (nullable = true)
 |-- YEARSEXP: string (nullable = true)
 |-- UNIVERSITY: string (nullable = true)
 |-- UOC_POSITION: string (nullable = true)
 |-- OTHER_POSITION: string (nullable = true)
 |-- OTHERSTATUS: string (nullable = true)
 |-- USERWIKI: string (nullable = true)
 |-- PU1: string (nullable = true)
 |-- PU2: string (nullable = true)
 |-- PU3: string (nullable = true)
 |-- PEU1: string (nullable = true)
 |-- PEU2: string (nullable = true)
 |-- PEU3: string (nullable = true)
 |-- ENJ1: string (nullable = true)
 |-- ENJ2: string (nullable = true)
 |-- Qu1: string (nullable = true)
 |-- Qu2: string (nullable = true)
 |-- Qu3: string (nullable = true)
 |-- Qu4: string (nullable = true)
 |-- Qu5: string (nullable = true)
 |-- Vis1: string (nullable = true)
 |-- Vis2: string (nullable = true)
 |-- Vis3: string (nullable = true)
 |-- Im1: string (nullable

In [5]:
# Registrando o DataFrame como um Temp Table
wiki4HE_DF.createOrReplaceTempView("wiki4HE_TB")
attributeInformation_DF.createOrReplaceTempView("attributeInformation_TB")

# Início da apresentação de análises

In [6]:
# Verificando número de linhas do Data Set [wiki4HE]
spSession.sql("SELECT COUNT(*) AS Amount FROM wiki4HE_TB").show()

+------+
|Amount|
+------+
|   913|
+------+



In [7]:
# Verificando a distribuição por Idade e Sexo
menDF = spSession.sql("SELECT AGE FROM wiki4HE_TB WHERE GENDER = 0")
womenDF = spSession.sql("SELECT AGE FROM wiki4HE_TB WHERE GENDER = 1")

Men = go.Histogram(x = menDF.toPandas()['AGE'], opacity = 0.75, name = 'Men')
Women = go.Histogram(x = womenDF.toPandas()['AGE'], opacity = 0.75, name = 'Women')

data = [Men, Women]
layout = go.Layout(barmode = 'overlay', title = "Histograma por Idade e Sexo do wiki4HE")

plotly.offline.iplot({
    "data": data,
    "layout": layout
})

In [8]:
# Agrupamento por área de atuação dos entrevistados
EDA1 = spSession.sql("SELECT att.DESCRIPTION AS Domain, COUNT(*) AS Amount " +
                     "FROM " +
                     "wiki4HE_TB wik JOIN attributeInformation_TB att " +
                     "  ON (wik.DOMAIN = att.ID AND att.COLUMN = 'DOMAIN') " +
                     "GROUP BY DESCRIPTION " +
                     "ORDER BY 2 DESC"
                    )
EDA1.show()

+--------------------+------+
|              Domain|Amount|
+--------------------+------+
|              Others|   361|
|   Arts & Humanities|   183|
|Engineering & Arc...|   137|
|      Law & Politics|   101|
|     Health Sciences|    73|
|            Sciences|    56|
|             Unknown|     2|
+--------------------+------+



In [9]:
# Representação gráfica do agrupamento por área de atuação 
data = [go.Bar(
            x = EDA1.toPandas()['Amount'],
            y = EDA1.toPandas()['Domain'],
            orientation = 'h'
)]

layout = go.Layout(title = "Domain")

plotly.offline.iplot({
    "data": data,
    "layout": layout
})

In [10]:
# Análise dos professores participantes: UOC_POSITION = 1 + OTHER_POSITION = 1 
EDA2 = spSession.sql("SELECT att1.DESCRIPTION AS Domain, att2.DESCRIPTION AS Gender, COUNT(*) AS Amount " +
                     "FROM " +
                     "wiki4HE_TB wik " + 
                     "  JOIN attributeInformation_TB att1 " +
                     "    ON (wik.DOMAIN = att1.ID AND att1.COLUMN = 'DOMAIN') " +
                     "  JOIN attributeInformation_TB att2 " +
                     "    ON (wik.GENDER = att2.ID AND att2.COLUMN = 'GENDER') " +
                     "WHERE wik.UOC_POSITION = 1 OR OTHER_POSITION = 1 " +
                     "GROUP BY att1.DESCRIPTION, att2.DESCRIPTION " +
                     "ORDER BY 3 DESC"
                    )
EDA2.show()

+--------------------+------+------+
|              Domain|Gender|Amount|
+--------------------+------+------+
|              Others|  Male|    67|
|              Others|Female|    54|
|Engineering & Arc...|  Male|    28|
|      Law & Politics|  Male|    26|
|   Arts & Humanities|Female|    26|
|      Law & Politics|Female|    18|
|            Sciences|  Male|    16|
|   Arts & Humanities|  Male|    13|
|     Health Sciences|Female|    12|
|     Health Sciences|  Male|     9|
|Engineering & Arc...|Female|     1|
|            Sciences|Female|     1|
+--------------------+------+------+



In [11]:
# Representação gráfica dos professores por Sexo
plotly.offline.iplot({
    "data": [go.Pie(labels = EDA2.toPandas()['Gender'], values = EDA2.toPandas()['Amount'])],
    "layout": go.Layout(title = "% de Professores por Sexo ")
})

In [12]:
# Análise gráfica da distribuição de professores por Domain e Sexo
EDA3 = spSession.sql("SELECT att1.DESCRIPTION AS Domain, COUNT(*) AS Amount " +
                     "FROM " +
                     "wiki4HE_TB wik " + 
                     "  JOIN attributeInformation_TB att1 " +
                     "    ON (wik.DOMAIN = att1.ID AND att1.COLUMN = 'DOMAIN') " +
                     "  JOIN attributeInformation_TB att2 " +
                     "    ON (wik.GENDER = att2.ID AND att2.COLUMN = 'GENDER') " +
                     "WHERE (wik.UOC_POSITION = 1 OR OTHER_POSITION = 1) " +
                     "  AND wik.GENDER = 0 " +
                     "GROUP BY att1.DESCRIPTION " +
                     "ORDER BY 2 DESC"
                    )

trace1 = go.Bar(
    x = EDA3.toPandas()['Domain'],
    y = EDA3.toPandas()['Amount'],
    name = 'Men'
)

EDA4 = spSession.sql("SELECT att1.DESCRIPTION AS Domain, COUNT(*) AS Amount " +
                     "FROM " +
                     "wiki4HE_TB wik " + 
                     "  JOIN attributeInformation_TB att1 " +
                     "    ON (wik.DOMAIN = att1.ID AND att1.COLUMN = 'DOMAIN') " +
                     "  JOIN attributeInformation_TB att2 " +
                     "    ON (wik.GENDER = att2.ID AND att2.COLUMN = 'GENDER') " +
                     "WHERE (wik.UOC_POSITION = 1 OR OTHER_POSITION = 1) " +
                     "  AND wik.GENDER = 1 " +
                     "GROUP BY att1.DESCRIPTION " +
                     "ORDER BY 2 DESC"
                    )


trace2 = go.Bar(
    x = EDA4.toPandas()['Domain'],
    y = EDA4.toPandas()['Amount'],
    name='Women'
)

data = [trace1, trace2]
layout = go.Layout(barmode = 'group', title = 'Distribuição de Professores por Domain e Sexo')

plotly.offline.iplot({
    "data": data,
    "layout": layout
})

In [13]:
# Analise gráfica das perguntas ENJ1 e ENJ2
EDA5 = spSession.sql("SELECT 'The use of Wikipedia<br>stimulates curiosity' AS ENJ, " +
                     "att.ID, att.DESCRIPTION AS Question, COUNT(*) AS Amount, " +
                     "ROUND(100*COUNT(*)/SUM(COUNT(*)) OVER(),2) AS Perc " +
                     "FROM " +
                     "wiki4HE_TB wik " + 
                     "  JOIN attributeInformation_TB att " +
                     "    ON (wik.ENJ1 = att.ID AND att.COLUMN = 'LikertScale') " +
                     "WHERE att.ID <> '?' " +
                     "GROUP BY att.ID, att.DESCRIPTION "
                    )

EDA6 = spSession.sql("SELECT 'The use of Wikipedia<br>is entertaining' AS ENJ, " +
                     "att.ID, att.DESCRIPTION AS Question, COUNT(*) AS Amount, " +
                     "ROUND(100*COUNT(*)/SUM(COUNT(*)) OVER(),2) AS Perc " +
                     "FROM " +
                     "wiki4HE_TB wik " + 
                     "  JOIN attributeInformation_TB att " +
                     "    ON (wik.ENJ2 = att.ID AND att.COLUMN = 'LikertScale') " +
                     "WHERE att.ID <> '?' " +
                     "GROUP BY att.ID, att.DESCRIPTION "
                    )

##################################################################################
top_labels = ['Strongly<br>agree', 'Agree', 'Neutral', 'Disagree',
              'Strongly<br>disagree']

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
          'rgba(190, 192, 213, 1)']

x_data1 = EDA5.toPandas()['Perc'].values.T.tolist()
x_data2 = EDA6.toPandas()['Perc'].values.T.tolist()
x_data = []
x_data.append(x_data1)
x_data.append(x_data2)

y_data1 = EDA5.toPandas()['ENJ'].values.T.tolist()
y_data2 = EDA6.toPandas()['ENJ'].values.T.tolist()
y_data = []
y_data.append(y_data1[0])
y_data.append(y_data2[0])

##################################################################################
traces = []

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x = [xd[i]],
            y = [yd],
            orientation = 'h',
            marker = dict(
                color = colors[i],
                line = dict(color = 'rgb(248, 248, 249)', width = 1)
            )
        ))

layout = go.Layout(
    xaxis = dict(
        showgrid = False,
        showline = False,
        showticklabels = False,
        zeroline = False,
        domain = [0.15, 1]
    ),
    yaxis = dict(
        showgrid = False,
        showline = False,
        showticklabels = False,
        zeroline = False,
    ),
    barmode = 'stack',
    paper_bgcolor = 'rgb(248, 248, 255)',
    plot_bgcolor = 'rgb(248, 248, 255)',
    margin = dict(l = 120, r = 10, t = 140, b = 80),
    showlegend = False,
)
##################################################################################

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref = 'paper', yref = 'y',
                            x = 0.14, y = yd,
                            xanchor = 'right',
                            text = str(yd),
                            font = dict(family = 'Arial', size = 14, color = 'rgb(67, 67, 67)'),
                            showarrow = False, align = 'right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref = 'x', yref = 'y',
                            x = xd[0] / 2, y = yd,
                            text = str(xd[0]) + '%',
                            font = dict(family = 'Arial', size = 14, color = 'rgb(248, 248, 255)'),
                            showarrow = False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref = 'x', yref = 'paper',
                                x = xd[0] / 2, y = 1.1,
                                text = top_labels[0],
                                font = dict(family = 'Arial', size = 14, color = 'rgb(67, 67, 67)'),
                                showarrow = False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref = 'x', yref = 'y',
                                    x = space + (xd[i]/2), y = yd, 
                                    text = str(xd[i]) + '%',
                                    font = dict(family = 'Arial', size = 14, color = 'rgb(248, 248, 255)'),
                                    showarrow = False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref = 'x', yref = 'paper',
                                        x = space + (xd[i]/2), y = 1.1,
                                        text = top_labels[i],
                                        font = dict(family = 'Arial', size = 14, color = 'rgb(67, 67, 67)'),
                                        showarrow = False))
            space += xd[i]

layout['annotations'] = annotations

##################################################################################
fig = go.Figure(data = traces, layout = layout)
plotly.offline.iplot(fig)

In [14]:
# Analise gráfica das perguntas ENJ1 e ENJ2
EDA6 = spSession.sql("SELECT 'ENJ1' AS ENJ, att1.DESCRIPTION AS Domain, COUNT(*) AS Amount " +
                     "FROM " +
                     "wiki4HE_TB wik " + 
                     "  JOIN attributeInformation_TB att1 " +
                     "    ON (wik.DOMAIN = att1.ID AND att1.COLUMN = 'DOMAIN') " +
                     "  JOIN attributeInformation_TB att2 " +
                     "    ON (wik.ENJ1 = att2.ID AND att2.COLUMN = 'LikertScale')" +
                     "GROUP BY att1.DESCRIPTION "
                    )

EDA7 = spSession.sql("SELECT 'ENJ2' AS ENJ, att1.DESCRIPTION AS Domain, COUNT(*) AS Amount " +
                     "FROM " +
                     "wiki4HE_TB wik " + 
                     "  JOIN attributeInformation_TB att1 " +
                     "    ON (wik.DOMAIN = att1.ID AND att1.COLUMN = 'DOMAIN') " +
                     "  JOIN attributeInformation_TB att2 " +
                     "    ON (wik.ENJ2 = att2.ID AND att2.COLUMN = 'LikertScale')" +
                     "GROUP BY att1.DESCRIPTION "
                    )

fig = {
  "data": [
    {
      "values": EDA6.toPandas()['Amount'].values.T.tolist(),
      "labels": EDA6.toPandas()['Domain'].values.T.tolist(),
      "domain": {"x": [0, .48]},
      "name": "ENJ1",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },
    {
      "values": EDA7.toPandas()['Amount'].values.T.tolist(),
      "labels": EDA7.toPandas()['Domain'].values.T.tolist(),
      "text":["ENJ2"],
      "textposition":"inside",
      "domain": {"x": [.52, 1]},
      "name": "ENJ2",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Distribuição por Domain nas perguntas ENJ1 e ENJ2<br>As respostas foram iguais",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "ENJ1",
                "x": 0.20,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "ENJ2",
                "x": 0.8,
                "y": 0.5
            }
        ]
    }
}
plotly.offline.iplot(fig)
##################################################################################

## FIM

#### Rômulo Leonardo Vieira da Silva
#### E-mail: romulo.leonardo@gmail.com
#### GitHub: github.com/romuloleo